<a href="https://colab.research.google.com/github/nephelim74/BeginProg/blob/main/spark_dz2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
# Создаем SparkSession
spark = SparkSession.builder \
    .appName("Employee Salary Comparison") \
    .getOrCreate()

In [3]:
# Данные
data = [
    (1, 'Joe', '70', '3'),
    (2, 'Henry', '80', '4'),
    (3, 'Sam', '60', None),
    (4, 'Max', '90', None)
]


In [4]:
# Создаем DataFrame с явным указанием схемы
columns = ['id', 'Name', 'Salary', 'ManagerId']
df = spark.createDataFrame(data, schema=columns)

In [5]:
# Преобразуем Salary в числовой формат
df = df.withColumn("Salary", df["Salary"].cast("int"))

In [6]:
# Печатаем исходный DataFrame
print("Исходный DataFrame:")
df.show()

Исходный DataFrame:
+---+-----+------+---------+
| id| Name|Salary|ManagerId|
+---+-----+------+---------+
|  1|  Joe|    70|        3|
|  2|Henry|    80|        4|
|  3|  Sam|    60|     NULL|
|  4|  Max|    90|     NULL|
+---+-----+------+---------+



In [9]:
# Объединяем таблицу с самой собой для сопоставления сотрудников и их менеджеров
joined_df = df.alias("employee") \
    .join(df.alias("manager"), F.col("employee.ManagerId") == F.col("manager.id"), "left") \
    .select(F.col("employee.Name").alias("EmployeeName"),
            F.col("employee.Salary").alias("EmployeeSalary"),
            F.col("manager.Name").alias("ManagerName"),
            F.col("manager.Salary").alias("ManagerSalary"))

In [10]:
# Фильтруем сотрудников, которые зарабатывают больше своих менеджеров
result_df = joined_df.filter(F.col("EmployeeSalary") > F.col("ManagerSalary")) \
    .select("EmployeeName")

In [11]:
# Печатаем результат
print("Сотрудники, которые зарабатывают больше своих менеджеров:")
result_df.show()

# Останавливаем SparkSession
spark.stop()

Сотрудники, которые зарабатывают больше своих менеджеров:
+------------+
|EmployeeName|
+------------+
|         Joe|
+------------+



### 1) Первым делом создается объект `SparkSession` с именем приложения "Employee Salary Comparison". `getOrCreate()` создает новую сессию
### 2)  Задается список кортежей, представляющих данные о сотрудниках: `(id, Name, Salary, ManagerId)`
### 3) Создается DataFrame `df` из данных `data`. `schema=columns` явно указывает имена столбцов
### 4) Столбец "Salary" преобразуется из строкового типа в целочисленный (`int`). Это необходимо для числовых сравнений.

### 5) Происходит соединение (`self-join`) DataFrame `df` с самим собой:
      * `df.alias("employee")`: DataFrame переименовывается в "employee".
      * `df.alias("manager")`: DataFrame переименовывается в "manager".
      * `.join(...)`: Объединение выполняется по условию `employee.ManagerId == manager.id` (здесь используется `F.col` для указания столбцов). `left` означает левое внешнее соединение, сохраняющее всех сотрудников, даже если у них нет менеджера.
      * `.select(...)`: Выбираются нужные столбцы и переименовываются для ясности.
Создается врт такая таблица, в которой строки включают связку по id и ManagerId:
Сотрудник и его менеджер

In [8]:
joined_df.show()

+------------+--------------+-----------+-------------+
|EmployeeName|EmployeeSalary|ManagerName|ManagerSalary|
+------------+--------------+-----------+-------------+
|         Joe|            70|        Sam|           60|
|       Henry|            80|        Max|           90|
|         Sam|            60|       NULL|         NULL|
|         Max|            90|       NULL|         NULL|
+------------+--------------+-----------+-------------+



### 6) Фильтрование `joined_df`, оставляя только строки, где зарплата сотрудника (`EmployeeSalary`) больше зарплаты его менеджера (`ManagerSalary`). Выбираются только имена сотрудников (`EmployeeName`).